In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2560362252305613356, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11446226341184017392
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys

import os
import numpy as np

import keras
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Activation, Dropout,Input, concatenate, Flatten  
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.applications.vgg19 import VGG19
from keras import optimizers, metrics, models, layers
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0
dim_ordering: tf


In [15]:
train_dir = 'data-ISBI-resized-final/train/'
validation_dir = 'data-ISBI-resized-final/validation/'
test_dir = 'data-ISBI-resized-final/test/'
test_augmented_dir = 'data-ISBI-resized-final/test-augmented/'

extracted_features_dir = "extracted_features/"

num_classes = 2
batch_size = 32
img_height, img_width = 380, 380
input_shape = (img_height, img_width, 3)
epochs = 25

In [6]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
#     validation_split= 0.25,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
#     subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
#     subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 59169 images belonging to 2 classes.
Found 18384 images belonging to 2 classes.
Found 966 images belonging to 2 classes.


In [7]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_VGG16_MobileNet_Concatenate3.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_VGG16_MobileNet_Concatenate3.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_VGG16_MobileNet_Concatenate3.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [8]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
         'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
#          'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
#          'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
         'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
         'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [12]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

    model = Sequential()
#     model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(num_classes, activation='softmax'))

    #     adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
    sgd_opt=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop_opt=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    
    model.compile(optimizer = sgd_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2,
                    callbacks=get_callbacks(params))
    
    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    print("_" * 30)
    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]

    print('Test Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))
    print("_" * 50)
    sys.stdout.flush() 
    keras.backend.clear_session()
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [13]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [16]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)
print(best)

Parameters testing:  {'lr': 1e-06, 'dropout': 0.6, 'momentum': 0.5, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.7123 - acc: 0.5179 - val_loss: 0.6673 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.6882 - acc: 0.5643 - val_loss: 0.6500 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 0.6724 - acc: 0.5974 - val_loss: 0.6399 - val_acc: 0.6806
Epoch 4/25
 - 6s - loss: 0.6640 - acc: 0.6145 - val_loss: 0.6339 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6338532870494354
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'lr': 1e-07, 'dropout': 0.5, 'momentum': 0.5, 'nesterov': False}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 6s - loss: 0.6845 - acc: 0.5577 - val_loss: 0.6679 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.6822 - acc: 0.56

 - 6s - loss: 0.7760 - acc: 0.4053 - val_loss: 0.7498 - val_acc: 0.3194
Epoch 4/25
 - 6s - loss: 0.7665 - acc: 0.4184 - val_loss: 0.7413 - val_acc: 0.3194
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 35us/step
Validation Accuracy: 31.9408%
Validation Loss: 0.7412817480254941
______________________________
Test Accuracy 0.3219461697722567
__________________________________________________
Parameters testing:  {'lr': 1e-06, 'dropout': 0.4, 'momentum': 0.9, 'nesterov': False}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 6s - loss: 0.7246 - acc: 0.4673 - val_loss: 0.6547 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.6462 - acc: 0.6639 - val_loss: 0.6313 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 0.6340 - acc: 0.6810 - val_loss: 0.6268 - val_acc: 0.6806
Epoch 4/25
 - 6s - loss: 0.6301 - acc: 0.6824 - val_loss: 0.6249 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy

 - 6s - loss: 0.6824 - acc: 0.5638 - val_loss: 0.6645 - val_acc: 0.6806
Epoch 7/25
 - 6s - loss: 0.6733 - acc: 0.5859 - val_loss: 0.6562 - val_acc: 0.6806
Epoch 8/25
 - 6s - loss: 0.6648 - acc: 0.6099 - val_loss: 0.6497 - val_acc: 0.6806
Epoch 00008: early stopping
18384/18384 [==============================] - 1s 35us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6496785635321528
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'lr': 0.001, 'dropout': 0.6, 'momentum': 0.9, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.5841 - acc: 0.7030 - val_loss: 0.5349 - val_acc: 0.7610
Epoch 2/25
 - 6s - loss: 0.5358 - acc: 0.7489 - val_loss: 0.5277 - val_acc: 0.7447
Epoch 3/25
 - 6s - loss: 0.5270 - acc: 0.7572 - val_loss: 0.5096 - val_acc: 0.7697
Epoch 4/25
 - 6s - loss: 0.5228 - acc: 0.7593 - val_loss: 0.5128 - val_acc: 0.7722
Epoch 5/25
 - 6s - loss: 

Epoch 00004: early stopping
18384/18384 [==============================] - 1s 36us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6229192908169188
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'lr': 0.001, 'dropout': 0.6, 'momentum': 0.9, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.5879 - acc: 0.6976 - val_loss: 0.5396 - val_acc: 0.7222
Epoch 2/25
 - 6s - loss: 0.5392 - acc: 0.7469 - val_loss: 0.5125 - val_acc: 0.7759
Epoch 3/25
 - 6s - loss: 0.5275 - acc: 0.7578 - val_loss: 0.5147 - val_acc: 0.7622
Epoch 4/25
 - 6s - loss: 0.5227 - acc: 0.7619 - val_loss: 0.5037 - val_acc: 0.7796
Epoch 5/25
 - 6s - loss: 0.5189 - acc: 0.7626 - val_loss: 0.5090 - val_acc: 0.7703
Epoch 6/25
 - 6s - loss: 0.5144 - acc: 0.7644 - val_loss: 0.5080 - val_acc: 0.7644
Epoch 7/25
 - 6s - loss: 0.5151 - acc: 0.7640 - val_loss: 0.4960 - val_acc: 0.7775
Epoch 00007: e

Epoch 00005: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6403475729043427
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'lr': 0.0001, 'dropout': 0.6, 'momentum': 0.9, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6200 - acc: 0.6789 - val_loss: 0.6029 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.5982 - acc: 0.6834 - val_loss: 0.5878 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 0.5829 - acc: 0.6864 - val_loss: 0.5729 - val_acc: 0.6807
Epoch 4/25
 - 6s - loss: 0.5699 - acc: 0.6965 - val_loss: 0.5591 - val_acc: 0.6998
Epoch 5/25
 - 6s - loss: 0.5579 - acc: 0.7167 - val_loss: 0.5495 - val_acc: 0.7020
Epoch 6/25
 - 6s - loss: 0.5477 - acc: 0.7345 - val_loss: 0.5376 - val_acc: 0.7407
Epoch 7/25
 - 6s - loss: 0.5391 - acc: 0.7466 - val_loss: 0.5296 - val_acc: 0.7635
Epoch 8/25
 -

...

Epoch 9/25
 - 6s - loss: 0.5605 - acc: 0.7426 - val_loss: 0.5359 - val_acc: 0.7490
Epoch 00009: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 74.8966%
Validation Loss: 0.5359426998801601
______________________________
Test Accuracy 0.7784679089026915
__________________________________________________
Parameters testing:  {'lr': 1e-06, 'dropout': 0.6, 'momentum': 0.5, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6632 - acc: 0.6111 - val_loss: 0.6336 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.6533 - acc: 0.6311 - val_loss: 0.6286 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 0.6513 - acc: 0.6412 - val_loss: 0.6256 - val_acc: 0.6806
Epoch 4/25
 - 6s - loss: 0.6494 - acc: 0.6478 - val_loss: 0.6237 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 38us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6236979813550845
__________________________

 - 7s - loss: 0.5845 - acc: 0.7011 - val_loss: 0.5365 - val_acc: 0.7739
Epoch 2/25
 - 6s - loss: 0.5382 - acc: 0.7480 - val_loss: 0.5074 - val_acc: 0.7774
Epoch 3/25
 - 6s - loss: 0.5217 - acc: 0.7614 - val_loss: 0.5002 - val_acc: 0.7788
Epoch 4/25
 - 6s - loss: 0.5164 - acc: 0.7650 - val_loss: 0.5042 - val_acc: 0.7722
Epoch 5/25
 - 6s - loss: 0.5156 - acc: 0.7648 - val_loss: 0.4983 - val_acc: 0.7769
Epoch 6/25
 - 6s - loss: 0.5117 - acc: 0.7670 - val_loss: 0.4991 - val_acc: 0.7775
Epoch 00006: early stopping
18384/18384 [==============================] - 1s 38us/step
Validation Accuracy: 77.7524%
Validation Loss: 0.49907159421421116
______________________________
Test Accuracy 0.8737060041407867
__________________________________________________
Parameters testing:  {'lr': 0.001, 'dropout': 0.5, 'momentum': 0.9, 'nesterov': False}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 6s - loss: 0.5767 - acc: 0.7084 - val_loss: 0.5598 - val_acc: 0.6973
Epoch 2/25
 - 6s - loss

Parameters testing:  {'lr': 0.001, 'dropout': 0.6, 'momentum': 0.9, 'nesterov': False}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 6s - loss: 0.5882 - acc: 0.6949 - val_loss: 0.5399 - val_acc: 0.7348
Epoch 2/25
 - 6s - loss: 0.5393 - acc: 0.7473 - val_loss: 0.5324 - val_acc: 0.7675
Epoch 3/25
 - 6s - loss: 0.5289 - acc: 0.7559 - val_loss: 0.5104 - val_acc: 0.7777
Epoch 4/25
 - 6s - loss: 0.5241 - acc: 0.7582 - val_loss: 0.5137 - val_acc: 0.7612
Epoch 5/25
 - 6s - loss: 0.5188 - acc: 0.7627 - val_loss: 0.4950 - val_acc: 0.7798
Epoch 6/25
 - 6s - loss: 0.5168 - acc: 0.7618 - val_loss: 0.5023 - val_acc: 0.7721
Epoch 7/25
 - 6s - loss: 0.5146 - acc: 0.7634 - val_loss: 0.5061 - val_acc: 0.7674
Epoch 8/25
 - 6s - loss: 0.5120 - acc: 0.7660 - val_loss: 0.5270 - val_acc: 0.7603
Epoch 00008: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 76.0281%
Validation Loss: 0.526969391927188
______________________________
Test Accuracy 0

Epoch 3/25
 - 6s - loss: 0.5201 - acc: 0.7643 - val_loss: 0.5073 - val_acc: 0.7715
Epoch 4/25
 - 6s - loss: 0.5141 - acc: 0.7665 - val_loss: 0.4966 - val_acc: 0.7787
Epoch 5/25
 - 6s - loss: 0.5129 - acc: 0.7664 - val_loss: 0.4968 - val_acc: 0.7773
Epoch 6/25
 - 6s - loss: 0.5093 - acc: 0.7686 - val_loss: 0.4934 - val_acc: 0.7804
Epoch 7/25
 - 6s - loss: 0.5074 - acc: 0.7680 - val_loss: 0.4972 - val_acc: 0.7758
Epoch 8/25
 - 6s - loss: 0.5066 - acc: 0.7708 - val_loss: 0.4939 - val_acc: 0.7802
Epoch 9/25
 - 6s - loss: 0.5050 - acc: 0.7704 - val_loss: 0.5051 - val_acc: 0.7669
Epoch 00009: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 76.6917%
Validation Loss: 0.5050733873800571
______________________________
Test Accuracy 0.8322981366459627
__________________________________________________
Parameters testing:  {'lr': 1e-05, 'dropout': 0.5, 'momentum': 0.3, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 

Test Accuracy 0.855072463768116
__________________________________________________
Parameters testing:  {'lr': 1e-05, 'dropout': 0.5, 'momentum': 0.9, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6397 - acc: 0.6686 - val_loss: 0.6243 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.6320 - acc: 0.6814 - val_loss: 0.6219 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 0.6267 - acc: 0.6825 - val_loss: 0.6194 - val_acc: 0.6806
Epoch 4/25
 - 6s - loss: 0.6229 - acc: 0.6830 - val_loss: 0.6172 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 38us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6171766321901864
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'lr': 0.001, 'dropout': 0.4, 'momentum': 0.7, 'nesterov': True}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6000 - acc: 0.683